# Use ONNX model converted from XGBoost with `ibm-watsonx-ai`
This notebook facilitates the `ONNX` format, the `XGBoost` library and the `watsonx.ai Runtime` service. It contains steps and code to work with [ibm-watsonx-ai](https://pypi.org/project/ibm-watsonx-ai/) library in order to convert the model to ONNX format. It also introduces commands necessary for trainining data, persisting and deploying the model and finally scoring it.

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Learning goals

The learning goals of this notebook are:
- Train an XGBoost model.
- Convert the XGBoost model to the ONNX format.
- Persist the converted model in the watsonx.ai repository.
- Deploy model for online scoring using the APIClient instance.
- Score sample records using the APIClient instance.


## Contents

This notebook contains the following parts:

1.	[Setting up the environment](#setup)<br>
2.	[Training an XGBoost model](#train-model)<br>
3.	[Converting the XGBoost model to the ONNX format](#convert)<br>
4.	[Persisting the converted ONNX model](#onnx-persist)<br>
5.	[Deploying and scoring the ONNX model](#deploy)<br>
6.  [Cleaning up](#cleanup)<br>
7.	[Summary and next steps](#summary)<br>


## 1. Setting up the environment <a id="setup"></a>
Before you use the sample code in this notebook, you must perform the following setup tasks:
- Create a [watsonx.ai Runtime](https://cloud.ibm.com/catalog/services/watsonxai-runtime) instance (information on service plans and further reading can be found [here](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp))

<a id="ml-packages"></a>
### 1.1. Installing and importing the `ibm-watsonx-ai` and dependencies 

Before you use the sample code in this notebook, install the following packages:

- ibm-watsonx-ai
- scikit-learn
- xgboost
- ONNX-related packages

**Note:** `ibm-watsonx-ai` documentation can be found [here](https://ibm.github.io/watsonx-ai-python-sdk/index.html).

In [ ]:
!pip install -U ibm-watsonx-ai | tail -n 1
!pip install scikit-learn==1.3.0 | tail -n 1
!pip install xgboost | tail -n 1

!pip install onnxruntime==1.16.3 | tail -n 1
!pip install onnx==1.16.2 | tail -n 1
!pip install onnxmltools | tail -n 1

In [90]:
import os
import json
import onnx
import onnxruntime
import getpass 

import pandas as pd 

from sklearn import datasets
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from ibm_watsonx_ai import Credentials, APIClient

from onnxconverter_common.data_types import FloatTensorType
from onnxmltools.convert import convert_xgboost
from onnxmltools.utils import save_model

### 1.2. Connecting to the watsonx.ai Runtime <a id="connect"></a>

Authenticate with the watsonx.ai Runtime service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```bash
    ibmcloud login
    ibmcloud iam api-key-create API_KEY_NAME
```

Get the value of `api_key` from the output.


Location of your watsonx.ai Runtime instance can be retrieved in the following way:
```bash
    ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
    ibmcloud resource service-instance INSTANCE_NAME
```

Get the value of `location` from the output.

**Tip**: You can generate your `Cloud API key` by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service-specific url by going to the [**Endpoint URLs** section of the watsonx.ai Runtime docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://cloud.ibm.com/catalog/services/watson-machine-learning" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance details.

You can also get the service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key, and paste it below.

**Action**: Enter your `api_key` and `location` in the following cells.

In [91]:
api_key = getpass.getpass("Please enter your api key (hit enter): ")

Please enter your api key (hit enter):  ········


In [127]:
location = "ENTER YOUR LOCATION HERE"

If you are running this notebook on Cloud, you can access the `location` via:

```python
    location = os.environ.get("RUNTIME_ENV_REGION")
```

In [96]:
url = f"https://{location}.ml.cloud.ibm.com"

In [97]:
credentials = Credentials(api_key=api_key, url=url)
client = APIClient(credentials=credentials)

### 1.3. Working with spaces <a id="spaces"></a>

First of all, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use the `ibm_watsonx_ai` SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

In [128]:
space_id = "ENTER YOUR SPACE ID HERE"

You can use the `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

To be able to interact with all resources available in watsonx.ai Runtime, you need to set space which you will be using.

In [101]:
client.set.default_space(space_id)

'SUCCESS'

## 2. Training the XGBoost model <a id="train-model"></a>

In this section, you will train a simple `XGBoost` model.

### 2.1 Loading the [iris](https://scikit-learn.org/1.3/auto_examples/datasets/plot_iris_dataset.html) data <a id="iris-load"></a>

In [102]:
iris = datasets.load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

### 2.2. Training the XGBoost model <a id="xgboost-train">

In [103]:
xgb_classifier = XGBClassifier(objective="objective='multiclass'")
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

### 2.3. Evaluating the model <a id="xgboost-test"></a>

In [118]:
xgb_classifier.predict(X_test[:3])

[0 0 2]


## 3. Converting the XGBoost model to the ONNX format <a id="convert"></a>

In this section, you will convert the model to the ONNX format.

### 3.1. Converting the model

In [105]:
initial_type = [("float_input", FloatTensorType([None, 4]))]
onnx_model = convert_xgboost(xgb_classifier, initial_types=initial_type)
onnx_model_filename = "xgboost_onnx_model.onnx"
save_model(onnx_model, onnx_model_filename)

**Note:** To convert a scikit-learn pipeline with an XGBoost model refer to the [documentation](https://onnx.ai/sklearn-onnx/auto_tutorial/plot_gexternal_xgboost.html).

<a id="onnx-check"></a>
### 3.2. Evaluating the ONNX model

After exporting the model, you should verify its integrity and ensure that it functions as expected. We will use `onnxruntime` to load the model and perform inference on the test data. Additionally, we’ll use `onnx`'s `checker` module to validate the exported ONNX model.

In [106]:
onnx.checker.check_model(onnx_model)

In [113]:
session = onnxruntime.InferenceSession(onnx_model_filename)

input_data = {session.get_inputs()[0].name: X_test[:3].tolist()}
session.run([], input_data)

[array([0, 0, 2], dtype=int64),
 array([[0.9935389 , 0.00468294, 0.00177807],
        [0.9935389 , 0.00468294, 0.00177807],
        [0.00128965, 0.00460506, 0.9941052 ]], dtype=float32)]

Every classifier, by design, produces two outputs: the predicted label and the probability distribution across all possible labels.

As you can see, the predicted values are consistent with those calculated in the [evaluation section](#xgboost-test).

<a id="onnx-persist"></a>
## 4. Persisting the converted ONNX model 

In this section, you will learn how to store your converted ONNX model in watsonx.ai Runtime.

Define model name, type and software spec.

In [108]:
sw_spec_id = client.software_specifications.get_id_by_name("onnxruntime_opset_19")
onnx_model_zip = "xgboost_onnx_model.zip"

Save the info to an archive.

In [109]:
!zip {onnx_model_zip} {onnx_model_filename}

  adding: xgboost_onnx_model.onnx (deflated 82%)


<a id="onnx-publish"></a>
### 4.1. Publishing the ONNX model to the watsonx.ai Runtime repository

In [110]:
metadata = {
    client.repository.ModelMetaNames.NAME: 'XGBoost to ONNX converted model',
    client.repository.ModelMetaNames.TYPE: 'onnxruntime_1.16',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_ID: sw_spec_id
}

published_model = client.repository.store_model(
    model=onnx_model_zip,
    meta_props=metadata
)

<a id="onnx-details"></a>
### 4.2. Getting model details

In [ ]:
published_model_id = client.repository.get_model_id(published_model)
model_details = client.repository.get_details(published_model_id)
print(json.dumps(model_details, indent=2))

<a id="deploy"></a>
## 5. Deploying and scoring the ONNX model
In this section you'll learn how to create an online scoring service and predict on unseen data.

<a id="onnx-create"></a>
### 5.1. Creating an online deployment for published model

In [112]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of XGBoost to ONNX converted model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_id, meta_props=metadata)



######################################################################################

Synchronous deployment creation for id: '7713bfd1-1b89-49bf-ab4b-1c709a420a0a' started

######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.
...
ready


-----------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_id='8795cc48-d9c7-466f-8295-8fc7462e8780'
-----------------------------------------------------------------------------------------------




In [114]:
deployment_id = client.deployments.get_id(created_deployment)

Now you can print an online scoring endpoint. 

In [ ]:
client.deployments.get_scoring_href(created_deployment)

<a id="deployment-details"> </a>
### 5.2. Getting deployment details

In [ ]:
client.deployments.get_details(deployment_id)

<a id="deployment-score"></a>
### 5.3. Scoring
You can use below method to do test scoring request against deployed model.
Prepare scoring payload with records to score.

In [115]:
scoring_payload = {
    "input_data": [
        {
            "values": X_test[:3]

        }
    ]
}

Use ``client.deployments.score()`` method to run scoring.

In [116]:
predictions = client.deployments.score(deployment_id, scoring_payload)

Let's see the predictions result

In [124]:
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "id": "label",
      "values": [
        0,
        0,
        2
      ]
    },
    {
      "id": "probabilities",
      "values": [
        [
          0.9935389161109924,
          0.004682942759245634,
          0.0017780718626454473
        ],
        [
          0.9935389161109924,
          0.004682942759245634,
          0.0017780718626454473
        ],
        [
          0.0012896523112431169,
          0.004605060908943415,
          0.9941052198410034
        ]
      ]
    }
  ]
}


As you can see, the predicted values are consistent with those calculated in the [evaluation section](#onnx-check).

<a id="cleanup"></a>
## 6. Cleaning up 
If you want to clean up after the notebook execution, i.e. remove any created assets like: 
- experiments<br>
- trainings<br>
- pipelines<br>
- model definitions<br>
- models<br>
- functions<br>
- deployments<br>

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/8e3a3ece6aa7d872eeb072d0705bc5a0cf90ff60/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).<br>

<a id="summary"></a>
## 7. Summary and next steps

You successfully completed this notebook! You learned how to use ONNX, XGBoost package as well as watsonx.ai Runtime for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=wx)_ for more samples, tutorials, documentation, how-tos, and blog posts.<br>

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.